In [1]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import re
from dotenv import load_dotenv

def clean_html_output(response):
    # Remove code block markers and extract JSON
    cleaned = re.sub(r"```html|```", "", response).strip()
    return cleaned

def updater(html_template, fields_dict):
  # Load environment variables
  load_dotenv()
  GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
  llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

  update_prompt = PromptTemplate(
              input_variables=["html_template", "fields_dict"],
              template="""
              You are a system tasked with intelligently updating a body of an HTML template with new data provided in a dictionary. The HTML template body will **not** contain explicit placeholders. Your goal is to understand the structure and content of the HTML body and integrate the data from the dictionary in a way that logically fits and enhances the information presented.

              **Instructions for Intelligent HTML body Update:**

              1.  **HTML body Understanding:** Analyze the provided HTML template body to understand its structure, identify key elements (e.g., headings, paragraphs, lists, tables, divs, spans), and infer the type of content each element is likely intended to hold. Consider the semantic meaning of HTML tags.

              2.  **Data Understanding:** Analyze the provided dictionary of data to understand the meaning of each key-value pair. Consider the data types and the likely real-world entities or attributes they represent.

              3.  **Intelligent Data Integration:** Based on your understanding of both the HTML body and the data, determine the most appropriate locations within the HTML to insert or update information from the data dictionary. This requires reasoning about the relationship between the data fields and the existing content or intended purpose of different HTML elements.

              4.  **Heuristic-Based Placement:** Employ reasonable heuristics to decide where the data fits best. Consider factors such as:
                  * **Semantic Similarity:** If a data key's name or the nature of its value seems related to the text content or the implied purpose of an HTML element, consider updating that element.
                  * **Data Type Appropriateness:** Insert numerical data into elements that might logically display numbers, text data into text-based elements, etc.
                  * **Contextual Relevance:** Ensure that the inserted data makes sense within the surrounding HTML structure and contributes to the overall information being presented.

              5.  **Output Generation:** The final output MUST be the updated HTML body string that incorporates the data from the dictionary in a meaningful and contextually appropriate way and do not add any additional text.
              
              Given the following HTML template:
              {html_template}
              
              And the following dictionary of data:
              {fields_dict}
              """
          )
# If a placeholder doesn't have a corresponding value (None), leave it unchanged.

  analysis_chain = LLMChain(llm=llm, prompt=update_prompt)

  llm_result = analysis_chain.run(html_template=html_template, fields_dict=fields_dict)
  
  llm_result = clean_html_output(llm_result)
  
  return llm_result

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import re
from dotenv import load_dotenv

def clean_html_output(response):
    # Remove code block markers and extract JSON
    cleaned = re.sub(r"```html|```", "", response).strip()
    return cleaned

def updater(html_template_body, fields_dict):
    # Load environment variables
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

    generated_html_body = ""
    continue_generation = True
    max_iterations = 10  # Safety limit
    iteration_count = 0

    while continue_generation and iteration_count < max_iterations:
        prompt_template = PromptTemplate(
            input_variables=["html_template", "fields_dict", "generated_html"],
            template="""
            You are tasked with generating the body content of an HTML template in a step-by-step manner. Your goal is to produce a complete and well-structured HTML body, intelligently integrating any relevant data provided in the dictionary. The HTML body should be generated in multiple chunks, with each chunk aiming to be under 200 lines.

            **Instructions for Step-by-Step HTML Body Generation (Chunked):**

            **Step 1: Analyze the HTML Body Template:** Carefully examine the structure and content of the provided HTML template body. Understand the overall layout and the types of information it seems to contain.

            **Step 2: Check Previously Generated Content:** Review the `generated_html` from previous steps.

            **Step 3: Determine Generation Start Point:**
            * **If no HTML body has been generated yet (`{{generated_html}}` is empty):** Begin generating the HTML body content from the beginning, aiming to create a complete and coherent structure.
            * **If some HTML body content has already been generated:** Compare the `{{generated_html}}` with the HTML template body. Identify the point where the generated content appears to have stopped in terms of completing the overall HTML body structure or integrating the necessary information.

            **Step 4: Continue Generation:** From the identified stopping point, generate the next logical chunk of the HTML body (not exceeding approximately 200 lines). Ensure this chunk seamlessly follows the previously generated content and contributes towards completing the entire HTML body.

            **Step 5: Intelligent Data Integration (If Applicable):** If the provided data dictionary contains information that logically fits within the current chunk of HTML being generated, integrate it intelligently.

            **Step 6: Heuristic-Based Placement:** Employ reasonable heuristics for data placement in each chunk.

            **Step 7: Output Generation (Chunked):** Generate the next logical chunk of the HTML body.

            **Step 8: Ensure Coherence:** Maintain the overall structure and coherence of the generated HTML body with the provided template (if it has initial content).

            **Step 9: Completion Signal:** Only include the clear closing marker 'END_GENERATION' in your output **when you have finished generating the entire HTML body content in a logical and complete manner.** Do not add any additional text outside the HTML body content.

            Given the following HTML template body:
            {html_template}

            And the following dictionary of data:
            {{
            {fields_dict}
            }}

            So far, the generated HTML body is:
            {generated_html}

            Generate the next chunk of the HTML body:
            """
        )

        analysis_chain = LLMChain(llm=llm, prompt=prompt_template)

        llm_result = analysis_chain.run(
            html_template=html_template_body,
            fields_dict=fields_dict,
            generated_html=generated_html_body
        )

        cleaned_chunk = clean_html_output(llm_result)
        generated_html_body += cleaned_chunk.strip() + "\n"

        if "END_GENERATION" in cleaned_chunk:
            continue_generation = False
            generated_html_body = generated_html_body.replace("END_GENERATION", "").strip()
            break

        iteration_count += 1

    return generated_html_body.strip()

In [95]:
file_path = r'D:\Programing\Work\Electro Pi\Landing pages\technology-marketing-business-portfolio-education\index.html'
with open(file_path, 'r', encoding='utf-8') as f:  # Specify UTF-8 encoding
                html_content = f.read()

In [96]:
def extract_html_body(html_content):
        """
        Extract the body content from an HTML string using regular expressions.
        
        Args:
            html_string (str): The full HTML content
        
        Returns:
            str: The extracted body content, or an empty string if no body is found
        """
        # Regex pattern to match body content
        # This handles various cases like different casing, whitespace, and attributes
        body_pattern = re.compile(r'<body[^>]*>(.*?)</body>', 
                                re.DOTALL | re.IGNORECASE)
        
        # Search for body content
        match = body_pattern.search(html_content)
        
        # Return matched body content or empty string
        return match.group(1).strip() if match else ''

In [97]:
html_body = extract_html_body(html_content)

In [98]:
print(html_body)

<!--=== Loader Start ======-->
<div id="loader-overlay">
  <div class="loader-wrapper">
    <div class="scoda-pulse"></div>
  </div>
</div>
<!--=== Loader End ======-->

<!--=== Wrapper Start ======-->
<div class="wrapper">

  <!--=== Header Start ======-->
  <nav class="navbar-scrollspy navbar navbar-default navbar-fixed white bootsnav on no-full navbar-transparent" data-minus-value-desktop="70" data-minus-value-mobile="55" data-speed="1000">
    <div class="container">
      <!--=== Start Atribute Navigation ===-->
      <div class="attr-nav">
        <ul class="social-media-dark social-top">
          <li><a href="#" class="icofont icofont-facebook"></a></li>
          <li><a href="#" class="icofont icofont-twitter"></a></li>
          <li><a href="#" class="icofont icofont-linkedin"></a></li>
        </ul>
      </div>
      <!--=== End Atribute Navigation ===-->

      <!--=== Start Header Navigation ===-->
      <div class="navbar-header">
        <button type="button" class="nav

In [99]:
electro_pi_data = {
    "page_title": "Electro Pi - AI-Powered Solutions for a Smarter Future",
    "company_name": "Electro Pi",
    "company_slogan": "Innovating with Intelligence",
    "slides": [
        {"title": "AI Solutions", "subtitle": "Cutting-Edge Development", "description": "Leveraging state-of-the-art AI technologies to transform industries."},
        {"title": "Business Automation", "subtitle": "Smarter Workflows"}
    ],
    "about_us_description": "We are an AI-driven technology company specializing in automation, data intelligence, and machine learning solutions. Our expertise spans across various industries, empowering businesses to optimize operations and enhance decision-making with innovative AI applications.",
    "what_we_do": {"title": "What We Do", "subtitle": "AI-Powered Transformation"},
    "services": [
        {"title": "AI-Powered Insights", "description": "Transforming raw data into actionable intelligence with AI-driven analytics."},
        {"title": "AI-Integrated Development", "description": "Developing AI-powered applications and systems for seamless automation."},
        {"title": "AI-Driven Marketing", "description": "Harnessing AI to personalize marketing strategies and enhance customer engagement."}
    ],
    "our_services": {"title": "Our Services", "subtitle": "Intelligent Solutions for a Digital World"},
    "service_details": [
        {"title": "AI-Enhanced Web Solutions", "description": "Creating intelligent web applications that adapt and learn from user interactions."},
        {"title": "AI-Driven App Development", "description": "Building smart applications with predictive analytics and automation capabilities."},
        {"title": "AI-Powered Digital Marketing", "description": "Using machine learning to optimize campaigns and maximize ROI."},
        {"title": "Smart UI/UX Design", "description": "Designing intuitive and adaptive user experiences powered by AI."}
    ],
    "team": {
        "title": "Meet Our Team",
        "subtitle": "Innovators in AI",
        "members": [
            {"name": "Esmail Essam Khalifa", "title": "AI Engineer"},
            {"name": "Ahmed Youssef", "title": "Data Scientist"},
            {"name": "Sarah Ibrahim", "title": "Machine Learning Engineer"},
            {"name": "Omar Hassan", "title": "CEO & Founder"}
        ]
    },
    "portfolio_categories": ["All", "AI Development", "Automation", "Predictive Analytics", "Computer Vision"],
    "portfolio_items": [
        {"title": "AI-Powered Chatbots", "category": "Natural Language Processing"},
        {"title": "Predictive Maintenance", "category": "Machine Learning"},
        {"title": "AI for Finance", "category": "Data Analytics"},
        {"title": "Computer Vision for Security", "category": "Computer Vision"},
        {"title": "AI-Powered Automation", "category": "Business Intelligence"},
        {"title": "Smart Healthcare Solutions", "category": "AI in Healthcare"}
    ],
    "stats": {
        "working_hours": {"count": 7500, "description": "AI-Driven Solutions Delivered"},
        "happy_clients": {"count": 300, "description": "Satisfied Clients"},
        "awards": {"count": 120, "description": "Industry Recognitions"},
        "projects_a_year": {"count": 500, "description": "AI Projects Annually"}
    },
    "testimonials": [
        {"name": "Michael Adams", "title": "CTO, Tech Innovators", "text": "Electro Pi revolutionized our workflow with AI automation, saving us time and resources."},
        {"name": "Sarah Collins", "title": "CEO, Data Insights", "text": "Their AI-driven analytics transformed how we handle business intelligence and decision-making."},
        {"name": "Kevin Hartman", "title": "Founder, AI Solutions", "text": "Electro Pi delivers top-notch AI solutions with an expert team and cutting-edge technology."}
    ],
    "pricing": {
        "title": "Our Pricing",
        "subtitle": "Choose Your AI Solution",
        "plans": [
            {"name": "Starter", "price": 49.99, "features": ["AI Chatbot Integration", "Basic Automation Tools", "1 User Access", "2TB Data Processing"]},
            {"name": "Business", "price": 99.99, "features": ["AI Chatbot + Analytics", "Advanced Workflow Automation", "5 Users", "5TB Data Processing"]},
            {"name": "Enterprise", "price": 199.99, "features": ["Custom AI Model Development", "Predictive Analytics Suite", "10 Users", "Full Data Security", "10TB Data Processing"]},
            {"name": "Ultimate AI", "price": 299.99, "features": ["Fully Custom AI Ecosystem", "Unlimited User Access", "Real-Time AI Insights", "20TB Data Processing"]}
        ]
    },
    "contact_us": {"title": "Contact Us", "subtitle": "Let’s Build the Future Together"},
    "footer_description": "Electro Pi specializes in AI-driven automation, predictive analytics, and machine learning solutions, empowering businesses with cutting-edge technologies.",
    "work_with_us": ["LinkedIn", "GitHub", "AI Community Forum", "TechCrunch", "Medium"],
    "useful_links": ["About Us", "AI Services", "Case Studies", "Privacy Policy", "Careers"],
    "contact": {
        "address": "AI Innovation Hub, Alexandria, Egypt",
        "phone": "+201206406887",
        "email": "contact@electropi.com",
        "website": "www.electropi.com"
    },
    "copyright_text": "© 2025 Electro Pi. All rights reserved",
    "cta": {
        "subtitle": "Ready to Innovate?",
        "title": "Start Your AI Journey",
        "description": "Partner with Electro Pi and unlock the full potential of AI for your business."
    }
}


In [100]:
respond = updater(html_body, electro_pi_data)

Iteration 1 completed.
Model indicated end of HTML body in iteration 1.


In [101]:
print(respond)

<!--=== Loader Start ======-->
<div id="loader-overlay">
  <div class="loader-wrapper">
    <div class="scoda-pulse"></div>
  </div>
</div>
<!--=== Loader End ======-->

<!--=== Wrapper Start ======-->
<div class="wrapper">

  <!--=== Header Start ======-->
  <nav class="navbar-scrollspy navbar navbar-default navbar-fixed white bootsnav on no-full navbar-transparent" data-minus-value-desktop="70" data-minus-value-mobile="55" data-speed="1000">
    <div class="container">
      <!--=== Start Atribute Navigation ===-->
      <div class="attr-nav">
        <ul class="social-media-dark social-top">
          <li><a href="#" class="icofont icofont-facebook"></a></li>
          <li><a href="#" class="icofont icofont-twitter"></a></li>
          <li><a href="#" class="icofont icofont-linkedin"></a></li>
        </ul>
      </div>
      <!--=== End Atribute Navigation ===-->

      <!--=== Start Header Navigation ===-->
      <div class="navbar-header">
        <button type="button" class="nav